In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install google-adk

In [19]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.genai import types
import requests
from google.adk.runners import Runner


print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [2]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")


✅ Helper functions defined.


In [20]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )
    

✅ Gemini API key setup complete.


In [17]:
retry_config=types.HttpRetryOptions( 
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)
print('configs checked!!')

configs checked!!


In [28]:
Bio_research_retriver = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction=""" You are a specialized scientific librarian and an internal tool. 
    Your sole and ONLY task is to EXECUTE THE SEARCH and populate the research_papers output key with
    a list of the 3 most relevant and recent scientific paper titles and their corresponding URLs.
    CRITICAL RULE: DO NOT provide any definition, summary, or explanation to the user. 
    DO NOT generate any text output that is not the execution of the search tool
    Your final output must be machine-readable text suitable for the next agent, formatted as a bulleted list of Title: [Link]""",
    tools = [google_search],
    output_key="research_papers",  # The result of this agent will be stored in the session state with this key.
)
print('agent1 created')

agent1 created


In [29]:
bio_summarizer = Agent(
    name = 'SummarizerAgent',
    model = Gemini(
        model = 'gemini-2.5-flash-lite',
        retry_options = retry_config
    ),
    instruction = """ u are a summarizing agent ,
    take the output from {research_papers} , Synthesize the most critical findings
    and produce a concise, well-written summary, followed by exactly 5 synthesized bullet points 
    that represent the most important conclusions of the overall research field 
    Add a bold heading before the main summary
    



    """,
    output_key = 'Summarized_papers'
)
print('agent 2 created')

agent 2 created


In [30]:
from google.adk.agents.sequential_agent import SequentialAgent
Root_agent = SequentialAgent(
    name ='coordinator',
    sub_agents = [Bio_research_retriver,bio_summarizer]
)
print('coordinator created')

coordinator created


In [31]:
APP_NAME = "BioResearchAI"
SESSION_ID = "user_session_01"
USER_ID = "user_666"


session_service = InMemorySessionService()

runner = Runner(
    agent=Root_agent,
    session_service=session_service,
    app_name="BioResearchAI"
)
print(' everything is fine here')

 everything is fine here


In [32]:
await run_session(
    runner,
     " what is CRISPR cas 9",
     "conversation03"

)








### Session: conversation03

User >  what is CRISPR cas 9
Model: > Title: CRISPR-Cas9: A Gene Editing Technology Causing a Major Upheaval in Biomedical Research
URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5098137/

Title: Recent Advances in CRISPR-Cas9 Genome Editing Technology for Biological and Biomedical Investigations
URL: https://pubmed.ncbi.nlm.nih.gov/28699397/

Title: Advances in CRISPR-Cas technology and its applications: revolutionising precision medicine
URL: https://www.frontiersin.org/articles/10.3389/fmed.2023.1165562/full
Model: > **CRISPR-Cas Technology: A Revolution in Gene Editing and Precision Medicine**

CRISPR-Cas technology has emerged as a transformative tool in biomedical research, fundamentally altering our ability to manipulate genetic material. This powerful gene-editing system, inspired by a natural bacterial defense mechanism, allows for precise and efficient modification of DNA sequences. Its versatility has opened up unprecedented avenues for under

In [ ]:
await run_session (
    runner,
    "so how is it difeerent from DNA",
    "comversation02"
)